In [5]:
!pip install requests beautifulsoup4 pandas

In [6]:
import requests
from bs4 import BeautifulSoup

url = "https://www.shl.com/solutions/products/product-catalog/"
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)

print(response.status_code)


200


In [7]:
soup = BeautifulSoup(response.text, "html.parser")

In [8]:
links = soup.find_all("a")
print(len(links))


213


In [9]:
for link in links:
    text = link.text.strip()
    href = link.get("href")
    if text and href and "product" in href:
        print(text)
        print(href)
        break

Products
/products/


In [10]:
for link in links:
    text = link.text.strip()
    href = link.get("href")
    if href and "/products/" in href and len(href) > 20:
        print(text)
        print("https://www.shl.com" + href)
        print("-----")

Occupational Personality Questionnaire (OPQ)
https://www.shl.com/products/assessments/personality-assessment/shl-occupational-personality-questionnaire-opq/
-----
Job-Focused Assessments (JFA)
https://www.shl.com/products/assessments/job-focused-assessments/
-----
Motivational Questionnaire (MQ)
https://www.shl.com/products/assessments/personality-assessment/shl-motivation-questionnaire-mq/
-----
Situational Judgment Tests (SJT)
https://www.shl.com/products/assessments/behavioral-assessments/situation-judgement-tests-sjt/
-----
SHL Verify
https://www.shl.com/products/assessments/cognitive-assessments/
-----
Assessments
https://www.shl.com/products/assessments/
-----
Behavioral Assessments
https://www.shl.com/products/assessments/behavioral-assessments/
-----
Cognitive Assessments
https://www.shl.com/products/assessments/cognitive-assessments/
-----
Personality Assessments
https://www.shl.com/products/assessments/personality-assessment/
-----
Video Interviews
https://www.shl.com/product

In [11]:
assessments = []
for link in links:
    name = link.text.strip()
    href = link.get("href")
    if href and "/products/" in href and len(href) > 20 and name:
        assessments.append({
            "name": name,
            "url": "https://www.shl.com" + href
        })
len(assessments)

51

In [12]:
assessments[:5]


[{'name': 'Occupational Personality Questionnaire (OPQ)',
  'url': 'https://www.shl.com/products/assessments/personality-assessment/shl-occupational-personality-questionnaire-opq/'},
 {'name': 'Job-Focused Assessments (JFA)',
  'url': 'https://www.shl.com/products/assessments/job-focused-assessments/'},
 {'name': 'Motivational Questionnaire (MQ)',
  'url': 'https://www.shl.com/products/assessments/personality-assessment/shl-motivation-questionnaire-mq/'},
 {'name': 'Situational Judgment Tests (SJT)',
  'url': 'https://www.shl.com/products/assessments/behavioral-assessments/situation-judgement-tests-sjt/'},
 {'name': 'SHL Verify',
  'url': 'https://www.shl.com/products/assessments/cognitive-assessments/'}]

In [13]:
import pandas as pd
df = pd.DataFrame(assessments)
df.to_csv("shl_assessments.csv", index=False)
df.head()

,name,url
0,Occupational Personality Questionnaire (OPQ),https://www.shl.com/products/assessments/perso...
1,Job-Focused Assessments (JFA),https://www.shl.com/products/assessments/job-f...
2,Motivational Questionnaire (MQ),https://www.shl.com/products/assessments/perso...
3,Situational Judgment Tests (SJT),https://www.shl.com/products/assessments/behav...
4,SHL Verify,https://www.shl.com/products/assessments/cogni...


In [14]:
df.shape

(51, 2)

In [15]:
!pip install sentence-transformers


In [16]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("all-MiniLM-L6-v2")
assessment_embeddings = model.encode(
    df["name"].tolist(),
    convert_to_tensor=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [42]:
from sentence_transformers import util
def recommend_assessments(query, top_k=5):
    query_embedding = model.encode(query, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, assessment_embeddings)[0]
    top_results = scores.topk(top_k)
    results = []
    for score, idx in zip(top_results.values, top_results.indices):
        results.append({
    "name": df.iloc[int(idx)]["Query"],
    "url": df.iloc[int(idx)]["Assessment_url"],
    "score": float(score)
})
    return results

In [18]:
query = "Looking for a software engineer with strong programming and problem solving skills"
recommend_assessments(query, top_k=5)

[{'name': 'Technical Skills',
  'url': 'https://www.shl.com/products/assessments/skills-and-simulations/technical-skills/',
  'score': 0.5039624571800232},
 {'name': 'Global Skills Development Report',
  'url': 'https://www.shl.com/products/product-catalog/view/global-skills-development-report/',
  'score': 0.3897325396537781},
 {'name': 'Skills & Simulations',
  'url': 'https://www.shl.com/products/assessments/skills-and-simulations/',
  'score': 0.38916832208633423},
 {'name': 'Business Skills',
  'url': 'https://www.shl.com/products/assessments/skills-and-simulations/business-skills/',
  'score': 0.3718205690383911},
 {'name': 'Apprentice + 8.0 Job Focused Assessment',
  'url': 'https://www.shl.com/products/product-catalog/view/apprentice-8-0-job-focused-assessment-4261/',
  'score': 0.32047104835510254}]

In [19]:
df = pd.read_csv("shl_assessments.csv")
assessment_embeddings = model.encode(df["name"].tolist(), convert_to_tensor=True)

In [20]:
!pip install fastapi uvicorn

In [21]:
from fastapi import FastAPI
from pydantic import BaseModel
app = FastAPI()
class QueryRequest(BaseModel):
    query: str
    top_k: int = 5
@app.get("/health")
def health():
    return {"status": "ok"}
@app.post("/recommend")
def recommend(req: QueryRequest):
    results = recommend_assessments(req.query, req.top_k)
    return {
        "query": req.query,
        "recommendations": results
    }

In [22]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class QueryRequest(BaseModel):
    query: str
    top_k: int = 5

@app.get("/health")
def health():
    return {"status": "ok"}

@app.post("/recommend")
def recommend(req: QueryRequest):
    results = recommend_assessments(req.query, req.top_k)
    return {
        "query": req.query,
        "recommendations": results
    }

In [23]:
from fastapi.testclient import TestClient
client = TestClient(app)
client.get("/health").json()


{'status': 'ok'}

In [24]:
client.post(
    "/recommend",
    json={
        "query": "Looking for a Java developer with teamwork skills",
        "top_k": 5
    }
).json()

{'query': 'Looking for a Java developer with teamwork skills',
 'recommendations': [{'name': 'Global Skills Development Report',
   'url': 'https://www.shl.com/products/product-catalog/view/global-skills-development-report/',
   'score': 0.44105109572410583},
  {'name': 'Technical Skills',
   'url': 'https://www.shl.com/products/assessments/skills-and-simulations/technical-skills/',
   'score': 0.4261664152145386},
  {'name': 'Skills & Simulations',
   'url': 'https://www.shl.com/products/assessments/skills-and-simulations/',
   'score': 0.3959016799926758},
  {'name': 'Job-Focused Assessments (JFA)',
   'url': 'https://www.shl.com/products/assessments/job-focused-assessments/',
   'score': 0.3901953101158142},
  {'name': 'Business Skills',
   'url': 'https://www.shl.com/products/assessments/skills-and-simulations/business-skills/',
   'score': 0.34096604585647583}]}

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

base_url = "https://www.shl.com"
catalog_base = "https://www.shl.com/solutions/products/product-catalog/"

headers = {"User-Agent": "Mozilla/5.0"}

all_assessments = []
seen_urls = set()
for page in range(1, 25):
    print(f"Scraping page {page}...")

    url = f"{catalog_base}?page={page}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Page failed, skipping")
        continue
    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.find_all("a")
    page_count = 0
    for link in links:
        name = link.text.strip()
        href = link.get("href")
        if (
            href
            and "/products/" in href
            and "/product-catalog/" not in href
            and len(href) > 25
            and name
        ):
            full_url = base_url + href

            if full_url not in seen_urls:
                seen_urls.add(full_url)
                all_assessments.append({
                    "name": name,
                    "url": full_url
                })
                page_count += 1
    print(f"  Found {page_count} new assessments")
    if page_count == 0:
        print("No more assessments found. Stopping.")
        break
    time.sleep(1)
print(f"\nTOTAL ASSESSMENTS COLLECTED: {len(all_assessments)}")

Scraping page 1...
  Found 15 new assessments
Scraping page 2...
  Found 0 new assessments
No more assessments found. Stopping.

TOTAL ASSESSMENTS COLLECTED: 15


In [26]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.shl.com"
catalog_url = "https://www.shl.com/solutions/products/product-catalog/"

headers = {"User-Agent": "Mozilla/5.0"}

resp = requests.get(catalog_url, headers=headers)
soup = BeautifulSoup(resp.text, "html.parser")

category_urls = set()

for a in soup.find_all("a", href=True):
    href = a["href"]
    if "/products/assessments/" in href:
        category_urls.add(base_url + href)

print("Category pages found:", len(category_urls))
category_urls=list(category_urls)
category_urls[:5]


Category pages found: 15


['https://www.shl.com/products/assessments/assessment-and-development-centers/',
 'https://www.shl.com/products/assessments/personality-assessment/',
 'https://www.shl.com/products/assessments/',
 'https://www.shl.com/products/assessments/behavioral-assessments/situation-judgement-tests-sjt/',
 'https://www.shl.com/products/assessments/behavioral-assessments/']

In [27]:
import pandas as pd

df = pd.read_excel("Gen_AI Dataset.xlsx")
df.head()


,Query,Assessment_url
0,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
1,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
2,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
3,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
4,I am hiring for Java developers who can also c...,https://www.shl.com/products/product-catalog/v...


In [28]:
import os
os.listdir()


['.config', 'Gen_AI Dataset.xlsx', 'shl_assessments.csv', 'sample_data']

In [29]:
import pandas as pd
df = pd.read_excel("Gen_AI Dataset.xlsx")
df.shape

(65, 2)

In [30]:
import pandas as pd

xls = pd.ExcelFile("Gen_AI Dataset.xlsx")
xls.sheet_names

['Train-Set', 'Test-Set']

In [31]:
import os

for root, dirs, files in os.walk("/"):
    for name in files:
        if "Gen_AI" in name:
            print(os.path.join(root, name))


/content/Gen_AI Dataset.xlsx


In [32]:
import os
os.listdir()


['.config', 'Gen_AI Dataset.xlsx', 'shl_assessments.csv', 'sample_data']

In [33]:
import pandas as pd

test_df = pd.read_excel("/content/Gen_AI Dataset.xlsx", sheet_name="Test-Set")
test_df.head()


,Query
0,Looking to hire mid-level professionals who ar...
1,Job Description\n\n Join a community that is s...
2,I am hiring for an analyst and wants applicati...
3,I have a JD Job Description\n\n People Science...
4,I am new looking for new graduates in my sales...


In [34]:
test_df.shape


(9, 1)

In [43]:
results = []

for idx, row in test_df.iterrows():
    query = row["Query"]   # column name

    recs = recommend_assessments(query, top_k=10)

    for r in recs:
        results.append({
            "query": query,
            "recommended_assessment": r["name"],
            "url": r["url"],
            "score": r["score"]
        })

final_df = pd.DataFrame(results)
final_df.head()


,query,recommended_assessment,url,score
0,Looking to hire mid-level professionals who ar...,KEY RESPONSIBITILES:\n\nManage the sound-scape...,https://www.shl.com/solutions/products/product...,0.546455
1,Looking to hire mid-level professionals who ar...,KEY RESPONSIBITILES:\n\nManage the sound-scape...,https://www.shl.com/solutions/products/product...,0.535649
2,Looking to hire mid-level professionals who ar...,Find me 1 hour long assesment for the below jo...,https://www.shl.com/solutions/products/product...,0.514787
3,Looking to hire mid-level professionals who ar...,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...,0.481217
4,Looking to hire mid-level professionals who ar...,I want to hire new graduates for a sales role ...,https://www.shl.com/solutions/products/product...,0.480810


In [38]:
df.columns


Index(['Query', 'Assessment_url'], dtype='object')

In [44]:
final_df.head()


,query,recommended_assessment,url,score
0,Looking to hire mid-level professionals who ar...,KEY RESPONSIBITILES:\n\nManage the sound-scape...,https://www.shl.com/solutions/products/product...,0.546455
1,Looking to hire mid-level professionals who ar...,KEY RESPONSIBITILES:\n\nManage the sound-scape...,https://www.shl.com/solutions/products/product...,0.535649
2,Looking to hire mid-level professionals who ar...,Find me 1 hour long assesment for the below jo...,https://www.shl.com/solutions/products/product...,0.514787
3,Looking to hire mid-level professionals who ar...,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...,0.481217
4,Looking to hire mid-level professionals who ar...,I want to hire new graduates for a sales role ...,https://www.shl.com/solutions/products/product...,0.480810


In [45]:
final_df.to_csv("final_predictions.csv", index=False)
print("✅ final_predictions.csv saved")


✅ final_predictions.csv saved
